In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [2]:
#https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min
dataset = pd.read_csv("data/high_diamond_ranked_10min.csv")

In [3]:
dataset.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [4]:
y = dataset.pop('blueWins')

In [5]:
dataset = dataset.drop(['gameId'], axis = 1)

0       4519157822
1       4523371949
2       4521474530
3       4524384067
4       4436033771
           ...    
9874    4527873286
9875    4527797466
9876    4527713716
9877    4527628313
9878    4523772935
Name: gameId, Length: 9879, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y, random_state=0)

In [50]:
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
scaler = MinMaxScaler().fit(dataset_dummies)
dataset_scaled = scaler.transform(dataset)

In [51]:
svm = SVC(C=100, gamma=0.001)
svm.fit(X_train_scaled, y_train)
print("Правильность метода опорных векторов на тестовом наборе: {}".format(svm.score(X_test_scaled, y_test)))

Правильность метода опорных векторов на тестовом наборе: 0.7226720647773279


In [59]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(), param_grid)
grid_search.fit(X_train_scaled, y_train)
print("Правильность на тестовом наборе: {:.2f}".format(grid_search.score(X_test_scaled, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Правильность на тестовом наборе: 0.72
Наилучшие значения параметров: {'C': 100, 'gamma': 0.001}


In [64]:
logReg = LogisticRegression(penalty="l1", solver='saga')
logReg.fit(X_train_scaled, y_train)
print("Правильность логистической регрессии на тестовом наборе: {}".format(logReg.score(X_test_scaled, y_test)))

Правильность логистической регрессии на тестовом наборе: 0.7234817813765182


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [48]:
pca = PCA(n_components=X_train_scaled.shape[1]//2)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
print(X_train_scaled.shape, X_train_pca.shape)

(7409, 38) (7409, 19)


In [49]:
logReg.fit(X_train_pca, y_train)
print("Правильность логистической регрессии на тестовом наборе: {}".format(logReg.score(X_test_pca, y_test)))
svm.fit(X_train_pca, y_train)
print("Правильность метода опорных векторов на тестовом наборе: {}".format(svm.score(X_test_pca, y_test)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Правильность логистической регрессии на тестовом наборе: 0.7170040485829959
Правильность метода опорных векторов на тестовом наборе: 0.7178137651821862


In [69]:
logScores = cross_val_score(logReg, dataset_scaled, y)
print("Результат перекрестной проверки: {}".format(logScores))
print("Средняя правильность перекрестной проверки: {}".format(logScores.mean()))
#svmScores = cross_val_score(svm, dataset_scaled, y)
#print("Средняя правильность перекрестной проверки: {}".format(svmScores.mean()))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Результат перекрестной проверки: [0.73497268 0.72760401 0.73693803]
Средняя правильность перекрестной проверки: 0.733171572563417


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))

In [73]:
forest = RandomForestClassifier(n_estimators=20)
forest.fit(X_train, y_train)
print("Правильность на обучающем наборе: {}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {}".format(forest.score(X_test, y_test)))

Правильность на обучающем наборе: 0.9968956674315023
Правильность на тестовом наборе: 0.7068825910931175


In [76]:
param_grid = {'n_estimators': [10, 20, 40, 60, 80, 100]}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid)
grid_search.fit(X_train, y_train)
print("Правильность на тестовом наборе: {}".format(grid_search.score(X_test, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Правильность на тестовом наборе: 0.7068825910931175
Наилучшие значения параметров: {'n_estimators': 80}
